In [10]:
from pathlib import Path
from datetime import datetime
import pandas as pd
from full_scanner import (
    get_sp500_tickers,
    get_nasdaq100_tickers,
    get_portfolio_tickers,
    scan_stocks,
    filter_buy_signals,
    create_excel_output,
    create_portfolio_excel,
    create_pdf_report
)

# Setup results directory
results_dir = Path.cwd() / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")

print("✓ Scanner modules loaded")
print(f"✓ Results directory: {results_dir}")
print(f"✓ Timestamp: {timestamp}")

✓ Scanner modules loaded
✓ Results directory: c:\workspace\my_script_project\scanner_results
✓ Timestamp: 20260106_2237


## Configuration

Set your scan parameters:
- **daily_bars**: Number of daily bars to analyze (default: 60)
- **weekly_bars**: Number of weekly bars to analyze (default: 52)
- **concurrency**: Number of parallel workers (default: 2, max recommended: 5)

In [11]:
# Scan parameters
DAILY_BARS = 60
WEEKLY_BARS = 52
CONCURRENCY = 2  # Be careful with higher values (rate limits)

print(f"Daily bars: {DAILY_BARS}")
print(f"Weekly bars: {WEEKLY_BARS}")
print(f"Concurrency: {CONCURRENCY}")

Daily bars: 60
Weekly bars: 52
Concurrency: 2


## Option 1: Scan Portfolio Stocks Only

Quick scan of your portfolio stocks from `stocks.txt`. Includes ALL signals (not just FULL HOLD + ADD).

In [12]:
print("Loading portfolio tickers from stocks.txt...")
portfolio_tickers = get_portfolio_tickers()

if portfolio_tickers:
    print(f"Found {len(portfolio_tickers)} portfolio stocks\n")

    # Run scan
    portfolio_results = scan_stocks(
        portfolio_tickers,
        category="Portfolio",
        daily_bars=DAILY_BARS,
        weekly_bars=WEEKLY_BARS,
        concurrency=CONCURRENCY
    )

    if not portfolio_results.empty:
        print(f"\n✓ Scanned {len(portfolio_results)} stocks")

        # Count by signal
        signal_counts = portfolio_results['signal'].value_counts()
        print("\nSignal breakdown:")
        for signal, count in signal_counts.items():
            print(f"  {signal}: {count}")

        # Save results - use create_portfolio_excel for all signals
        xlsx_path = results_dir / f'portfolio_scanner_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_portfolio_{timestamp}.pdf'

        create_portfolio_excel(portfolio_results, xlsx_path, category="Portfolio")
        create_pdf_report(portfolio_results, portfolio_results, pdf_path, timestamp, category="Portfolio")

        print(f"\n✓ Excel: {xlsx_path.name}")
        print(f"✓ PDF: {pdf_path.name}")

        # Show FULL HOLD + ADD stocks
        buy_signals = filter_buy_signals(portfolio_results, 'FULL HOLD + ADD')
        if not buy_signals.empty:
            print(f"\n🎯 {len(buy_signals)} FULL HOLD + ADD signals:")
            # Display only columns that exist
            display_cols = ['ticker', 'signal', 'current_price']
            print(buy_signals[display_cols].to_string(index=False))
        else:
            print("\n⚠️ No FULL HOLD + ADD signals in portfolio")
else:
    print("⚠️ No portfolio tickers found in stocks.txt")

Loading portfolio tickers from stocks.txt...
âœ“ Loaded 13 portfolio tickers from stocks.txt

Found 13 portfolio stocks

ðŸ” Scanning 13 Portfolio stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
âœ“ [1/13] TSLA   -> FULL HOLD + ADD      $431.84
âœ“ [2/13] NVDA   -> FULL HOLD + ADD      $187.00
âœ“ [5/13] PLTR   -> FULL HOLD + ADD      $179.57
âœ“ [7/13] ASML   -> FULL HOLD + ADD      $1240.81
âœ“ [11/13] MRVL   -> FULL HOLD + ADD      $87.81

âœ“ Scan complete: 13 analyzed, 5 FULL HOLD + ADD signals found


✓ Scanned 13 stocks

Signal breakdown:
  FULL HOLD + ADD: 5
  HOLD MOST + REDUCE: 4
  CASH: 2
  HOLD: 2
âœ“ Excel file created: c:\workspace\my_script_project\scanner_results\portfolio_scanner_20260106_2237.xlsx
  - All: 13 stocks
  - FULL HOLD + ADD: 5 stocks
  - HOLD MOST â†’ REDUCE: 0 stocks
  - HOLD: 2 stocks
  - CASH: 2 stocks
âœ“ PDF report created: c:\workspace\my_script_project\scanner_results\scanner_report_portfolio_20260106_22

## Option 2: Scan S&P 500

Full scan of ~500 stocks. Takes 5-10 minutes depending on cache. Only outputs FULL HOLD + ADD signals.

In [ ]:
print("Fetching S&P 500 ticker list...")
sp500_tickers = get_sp500_tickers()
print(f"Found {len(sp500_tickers)} S&P 500 stocks\n")

# Run scan
sp500_results = scan_stocks(
    sp500_tickers,
    category="S&P 500",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not sp500_results.empty:
    # Filter for buy signals only
    sp500_buy = filter_buy_signals(sp500_results, 'FULL HOLD + ADD')
    print(f"\n🎯 S&P 500: {len(sp500_buy)} FULL HOLD + ADD signals found")

    if not sp500_buy.empty:
        # Save results
        xlsx_path = results_dir / f'sp500_analysis_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_sp500_{timestamp}.pdf'

        create_excel_output(sp500_buy, xlsx_path, category="S&P 500")
        create_pdf_report(sp500_buy, sp500_results, pdf_path, timestamp, category="S&P 500")

        print(f"\n✓ Excel: {xlsx_path.name}")
        print(f"✓ PDF: {pdf_path.name}")

        # Display top results
        print("\nTop results:")
        display_cols = ['ticker', 'signal', 'current_price']
        print(sp500_buy[display_cols].head(10).to_string(index=False))
    else:
        print("\n⚠️ No FULL HOLD + ADD signals found in S&P 500")

Fetching S&P 500 ticker list...
âœ“ Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

ðŸ” Scanning 503 S&P 500 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 2 threads
âœ“ [4/503] ABBV   -> FULL HOLD + ADD      $223.93
âœ“ [9/503] AFL    -> FULL HOLD + ADD      $111.96
âœ“ [14/503] ALB    -> FULL HOLD + ADD      $158.15
âœ“ [19/503] ALL    -> FULL HOLD + ADD      $207.92
âœ“ [20/503] GOOGL  -> FULL HOLD + ADD      $314.34
âœ“ [21/503] GOOG   -> FULL HOLD + ADD      $314.55
âœ“ [23/503] AMZN   -> FULL HOLD + ADD      $240.93
âœ“ [27/503] AXP    -> FULL HOLD + ADD      $383.56
âœ“ [28/503] AIG    -> FULL HOLD + ADD      $78.06
âœ“ [32/503] AME    -> FULL HOLD + ADD      $214.16
âœ“ [33/503] AMGN   -> FULL HOLD + ADD      $330.17
âœ“ [34/503] APH    -> FULL HOLD + ADD      $141.38
âœ“ [35/503] ADI    -> FULL HOLD + ADD      $292.94
âœ“ [38/503] APO    -> FULL HOLD + ADD      $152.70
âœ“ [39/503] AAPL   -> FULL HOLD + ADD      $262.36
âœ“ [40/503] A

## Option 3: Scan NASDAQ 100

Scan ~100 tech-heavy stocks. Takes 2-5 minutes. Only outputs FULL HOLD + ADD signals.

In [ ]:
print("Fetching NASDAQ 100 ticker list...")
nasdaq100_tickers = get_nasdaq100_tickers()
print(f"Found {len(nasdaq100_tickers)} NASDAQ 100 stocks\n")

# Run scan
nasdaq100_results = scan_stocks(
    nasdaq100_tickers,
    category="NASDAQ 100",
    daily_bars=DAILY_BARS,
    weekly_bars=WEEKLY_BARS,
    concurrency=CONCURRENCY
)

if not nasdaq100_results.empty:
    # Filter for buy signals only
    nasdaq100_buy = filter_buy_signals(nasdaq100_results, 'FULL HOLD + ADD')
    print(f"\n🎯 NASDAQ 100: {len(nasdaq100_buy)} FULL HOLD + ADD signals found")

    if not nasdaq100_buy.empty:
        # Save results
        xlsx_path = results_dir / f'nasdaq100_analysis_{timestamp}.xlsx'
        pdf_path = results_dir / f'scanner_report_nasdaq100_{timestamp}.pdf'

        create_excel_output(nasdaq100_buy, xlsx_path, category="NASDAQ 100")
        create_pdf_report(nasdaq100_buy, nasdaq100_results, pdf_path, timestamp, category="NASDAQ 100")

        print(f"\n✓ Excel: {xlsx_path.name}")
        print(f"✓ PDF: {pdf_path.name}")

        # Display top results
        print("\nTop results:")
        display_cols = ['ticker', 'signal', 'current_price']
        print(nasdaq100_buy[display_cols].head(10).to_string(index=False))
    else:
        print("\n⚠️ No FULL HOLD + ADD signals found in NASDAQ 100")

## Quick Analysis: View Recent Results

Load and analyze previously generated results without re-running scans.

In [ ]:
# List recent scanner results
import os

print("Recent scanner results:\n")
files = sorted(results_dir.glob('*.xlsx'), key=os.path.getmtime, reverse=True)

if files:
    for i, f in enumerate(files[:5], 1):
        size_kb = f.stat().st_size / 1024
        mod_time = datetime.fromtimestamp(f.stat().st_mtime).strftime("%Y-%m-%d %H:%M")
        print(f"{i}. {f.name} ({size_kb:.1f} KB) - {mod_time}")
else:
    print("No scanner results found yet. Run a scan above!")

## Load and View a Specific Result

Replace the filename below with one from the list above.

In [ ]:
# Load a specific result file
filename = 'portfolio_scanner_20260106_2232.xlsx'  # Change this to your file
filepath = results_dir / filename

if filepath.exists():
    # Load all sheets
    xl_file = pd.ExcelFile(filepath)
    print(f"Sheets in {filename}:")
    for sheet in xl_file.sheet_names:
        df = pd.read_excel(filepath, sheet_name=sheet)
        print(f"\n{sheet}: {len(df)} stocks")
        if not df.empty and len(df) > 0:
            # Show columns available
            display_cols = [c for c in ['ticker', 'signal', 'current_price'] if c in df.columns]
            print(df[display_cols].head(5).to_string(index=False))
else:
    print(f"File not found: {filename}")